<a href="https://colab.research.google.com/github/Emad78/Emotion-detection/blob/main/emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dadmatools
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.6/862.6 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd
import pickle
import fasttext

from tensorflow.keras import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Input, Embedding, Flatten, SimpleRNN
from tensorflow.keras.optimizers import SGD
from tensorflow import stack
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,  accuracy_score
from dadmatools.embeddings import get_embedding, get_all_embeddings_info, get_embedding_info
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier



In [ ]:
data = pd.read_csv("drive/MyDrive/Work/CleanData_arman.csv")
data = data[['text', 'label']]

train, test = train_test_split(data, test_size=0.3)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)


In [ ]:
train = train[train.label != 'OTHER']
test = test[test.label != 'OTHER']

In [ ]:
text = ''
for i in range(len(train)):
  text += train.loc[i, 'text'] + " __label__"+train.loc[i, 'label'] + "\n"

with open("drive/MyDrive/Work/train_fasttext.txt", 'w', encoding='UTF-8') as file:
  file.write(text)

In [ ]:
fast_text0 = fasttext.FastText.train_supervised("drive/MyDrive/Work/train_fasttext.txt", lr=0.5, epoch=25, wordNgrams=3, bucket=200000, dim=100, loss='ova')
fast_text0.save_model("drive/MyDrive/Work/fast_text0.model")
fast_text1 = get_embedding('fasttext-commoncrawl-bin')
fast_text1.model.save_model('drive/MyDrive/Work/fast_text1.model')


# NN Model

In [ ]:

def NN_prepare_data(train, temp):
  enc = LabelBinarizer()
  test, valid = train_test_split(temp, test_size=0.5)

  x_train = np.array(train['text'].apply(fast_text0.get_sentence_vector))
  x_test = np.array(test['text'].apply(fast_text0.get_sentence_vector))
  x_val = np.array(valid['text'].apply(fast_text0.get_sentence_vector))

  x_train1 = np.array(train['text'].apply(fast_text1.embedding_text))
  x_test1 = np.array(test['text'].apply(fast_text1.embedding_text))
  x_val1 = np.array(valid['text'].apply(fast_text1.embedding_text))

  for i in range(len(x_train)):
    x_train[i] = np.concatenate((x_train[i], x_train1[i]), axis=0)

  for i in range(len(x_test)):
    x_test[i] = np.concatenate((x_test[i], x_test1[i]), axis=0)

  for i in range(len(x_val)):
    x_val[i] = np.concatenate((x_val[i], x_val1[i]), axis=0)


  enc.fit(train['label'])
  y_train = enc.transform(train['label'])
  y_test = enc.transform(test['label'])
  y_val = enc.transform(valid['label'])

  return x_train, y_train, x_test, y_test, x_val, y_val, enc

In [ ]:
def create_model(input_size, output_size):
  model = Sequential()
  model.add(Input(input_size))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(output_size, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
  return model

In [ ]:
x, y, x_test, y_test, x_val, y_val, enc = prepare_data(train, test)

In [ ]:
model = create_model(400, 6)
print(model.summary())
model.fit(stack(x),y, epochs=80, batch_size=128, validation_data=(stack(x_val), y_val))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                4010      
                                                                 
 dense_3 (Dense)             (None, 6)                 66        
                                                                 
Total params: 4,076
Trainable params: 4,076
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/80
30/30 [==============================] - 1s 8ms/step - loss: 1.7401 - acc: 0.4201 - val_loss: 1.7171 - val_acc: 0.5104
Epoch 2/80
30/30 [==============================] - 0s 4ms/step - loss: 1.6028 - acc: 0.7225 - val_loss: 1.6319 - val_acc: 0.5031
Epoch 3/80
30/30 [==============================] - 0s 4ms/step - loss: 1.4488 - acc: 0.7376 - val_loss: 1.5534 - val_acc: 0.5117
Epoch 4/80
30/30 [==============================] - 0

In [ ]:
from tensorflow import one_hot, argmax
pre = model.predict(stack(x_test))
y_pred = []
for p in pre:
  s = len(p)*[0]
  s[argmax(p)] = 1
  y_pred.append(s)

y_pred = enc.inverse_transform(np.array(y_pred))
actual = enc.inverse_transform(y_test)
model.evaluate(stack(x_test), y_test)


26/26 [==============================] - 0s 2ms/step - loss: 1.2097 - acc: 0.6638


[1.2097331285476685, 0.6638036966323853]

In [ ]:
print(classification_report(actual, y_pred))

              precision    recall  f1-score   support

       ANGRY       0.71      0.74      0.72       162
        FEAR       0.81      0.80      0.81       112
       HAPPY       0.53      0.68      0.59       117
        HATE       0.71      0.68      0.70        84
         SAD       0.65      0.61      0.63       201
    SURPRISE       0.70      0.58      0.63       139

    accuracy                           0.67       815
   macro avg       0.69      0.68      0.68       815
weighted avg       0.68      0.67      0.68       815



In [ ]:
with open('drive/MyDrive/Work/6class_NN.model', 'wb') as file:
  pickle.dump(model, file)

# classic

In [ ]:

def prepare_data(train, test):
  enc = LabelBinarizer()
  
  x_train = list(train['text'].apply(fast_text0.get_sentence_vector))
  x_test = list(test['text'].apply(fast_text0.get_sentence_vector))
  
  x_train1 = list(train['text'].apply(fast_text1.embedding_text))
  x_test1 = list(test['text'].apply(fast_text1.embedding_text))
  
  for i in range(len(x_train)):
    x_train[i] = list(np.concatenate((x_train[i], x_train1[i]), axis=0))

  for i in range(len(x_test)):
    x_test[i] = list(np.concatenate((x_test[i], x_test1[i]), axis=0))


  y_train = (train['label'])
  y_test = (test['label'])

  return x_train, y_train, x_test, y_test

In [ ]:
x, y, x_test, y_test = prepare_data(train, test)

In [ ]:
svm = SVC(kernel='rbf')
svm.fit(x, y)
pre = svm.predict(x_test)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(x, y)
pre = knn.predict(x_test)

In [ ]:
mlp = MLPClassifier(random_state=10, max_iter=1000)
mlp.fit(x, y)
pre = mlp.predict(x_test)


In [ ]:
print(classification_report(y_test, pre))

              precision    recall  f1-score   support

       ANGRY       0.63      0.61      0.62       335
        FEAR       0.80      0.79      0.80       235
       HAPPY       0.53      0.54      0.54       257
        HATE       0.69      0.64      0.66       176
       OTHER       0.37      0.52      0.43       320
         SAD       0.60      0.51      0.55       376
    SURPRISE       0.57      0.45      0.51       252

    accuracy                           0.57      1951
   macro avg       0.60      0.58      0.59      1951
weighted avg       0.59      0.57      0.57      1951



In [ ]:
accuracy_score(y_test, pre)

0.6705521472392638

In [ ]:
with open('drive/MyDrive/Work/6class_svm.model', 'wb') as file:
  pickle.dump(svm, file)
with open('drive/MyDrive/Work/6class_mlp.model', 'wb') as file:
  pickle.dump(mlp, file)
with open('drive/MyDrive/Work/6class_knn.model', 'wb') as file:
  pickle.dump(knn, file)